In [5]:
import os, sys

project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
os.environ['PYTHONPATH'] = project_root
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print("PYTHONPATH manually set to:", os.environ['PYTHONPATH'])

PYTHONPATH manually set to: c:\Users\ndhaj\Desktop


In [6]:
import numpy as np
from numpy.random import default_rng
import gpie
gpie.set_backend(np)
from gpie import Graph, SparsePrior, GaussianMeasurement, fft2, mse
from gpie.core.linalg_utils import random_binary_mask

# 1. パラメータ設定
shape = (256, 256)
rho = 0.1          # sparsity
var = 1e-4         # noise variance
mask_ratio = 0.3  # percentage of observed entries

# 2. ユニタリ行列 U の生成
rng = default_rng(seed=12)
mask = random_binary_mask(shape, subsampling_rate=mask_ratio, rng = rng)

# 3. グラフ構築
class CompressiveSensingGraph(Graph):
    def __init__(self, shape, rho):
        super().__init__()
        x = ~SparsePrior(rho=rho, shape=shape, damping = 0.03, label = "x", dtype = np.complex64)
        with self.observe():
            GaussianMeasurement(var=var, mask=mask) @ (fft2(x))
        self.compile()

g = CompressiveSensingGraph(rho = rho, shape = shape)

# 4. RNG設定（初期メッセージとサンプリングを分離）
g.set_init_rng(np.random.default_rng(seed=11))
g.generate_sample(rng=np.random.default_rng(seed=42))

# 5. 真の信号と観測データを取得
true_x = g.get_wave("x").get_sample()

In [7]:
# 6. 推論の実行
def monitor(graph, t):
    x = graph.get_wave("x")
    est = x.compute_belief().data
    err = mse(est, true_x)
    if t % 5 == 0:
        print(f"[t={t}] MSE = {err:.5e}")

g.run(n_iter=30, callback=monitor)

# 8. 最終精度を表示
final_est = g.get_wave("x").compute_belief().data
print(f"Final MSE after 30 iterations: {mse(final_est, true_x):.5e}")

[t=0] MSE = 4.28132e-01
[t=5] MSE = 4.93058e-03
[t=10] MSE = 5.68045e-05
[t=15] MSE = 4.41228e-05
[t=20] MSE = 4.41061e-05
[t=25] MSE = 4.41046e-05
Final MSE after 30 iterations: 4.41046e-05
